In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
...

driver = webdriver.Chrome('C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe')
driver.get('https://www.youtube.com/channel/UCI7odJRqHJxb_1d4L7bzcrw/videos')

#여기서부터
num_of_pagedowns=100
body = driver.find_element_by_xpath('/html/body')

#자동으로 스크롤을 내려서 더이상 영상이 로드 안될때까지 내려줌.
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)
    num_of_pagedowns -= 1
    try:
        driver.find_element_by_xpath("""//*[@id="feed-main-what_to_watch"]/button""").click()
    except:
        None
        
page = driver.page_source
page

'<html style="font-size: 10px;font-family: Roboto, Arial, sans-serif; " lang="ko-KR"><head><script data-original-src="/yts/jsbin/player_ias-vfl8EyRMW/ko_KR/miniplayer.js" src="/yts/jsbin/player_ias-vfl8EyRMW/ko_KR/miniplayer.js"></script><script data-original-src="/yts/jsbin/player_ias-vfl8EyRMW/ko_KR/remote.js" src="/yts/jsbin/player_ias-vfl8EyRMW/ko_KR/remote.js"></script><meta http-equiv="origin-trial" data-feature="Web Components V0" data-expires="2020-10-23" content="AhbmRDASY7NuOZD9cFMgQihZ+mQpCwa8WTGdTx82vSar9ddBQbziBfZXZg+ScofvEZDdHQNCEwz4yM7HjBS9RgkAAABneyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJXZWJDb21wb25lbnRzVjAiLCJleHBpcnkiOjE2MDM0ODY4NTYsImlzU3ViZG9tYWluIjp0cnVlfQ=="><meta http-equiv="origin-trial" data-feature="Web Components V0" data-expires="2020-10-27" content="Av2+1qfUp3MwEfAFcCccykS1qFmvLiCrMZ//pHQKnRZWG9dldVo8HYuJmGj2wZ7nDg+xE4RQMQ+Ku1zKM3PvYAIAAABmeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IldlYkNvbXBvbmVudHNWMCIsImV4cGlyeSI6M

In [ ]:
from bs4 import BeautifulSoup
#태그, 클래스
soup = BeautifulSoup(page,'lxml')

all_title = soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')
title = [soup.find_all('a','yt-simple-endpoint style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(all_title))]
title

In [ ]:
from bs4 import BeautifulSoup

...


soup = BeautifulSoup(page,'lxml')

#채널명('태그','class명')
chennel = soup.find('yt-formatted-string','style-scope ytd-channel-name').string
#구독자 수
sub_num = soup.find('yt-formatted-string','style-scope ytd-c4-tabbed-header-renderer').string
#조회수, 올린지 얼마나 되었는지(업로드 시점)
c = soup.find_all('span','style-scope ytd-grid-video-renderer')
view_num = [soup.find_all('span','style-scope ytd-grid-video-renderer')[n].string for n in range(0,len(c))]
view_num

In [ ]:
from time import localtime, strftime

...

#현재 시간
extract_date = strftime("%Y/%m/%d %H:%M:%S", localtime())
extract_date

In [ ]:
...

youtube_video_list = []

x = 0		#조회수의 index
y = 1		#업로드 시점의 index

for i in range(0,len(all_title)):
    roww = []
    roww.append(title[i])
#     roww.append(video_time[i].strip())
    roww.append(chennel)
    roww.append(sub_num)
    roww.append(view_num[x])
    x += 2		#조회수만 append
    roww.append(view_num[y])
    y += 2		#업로드 시점만 append
    roww.append(extract_date)
    youtube_video_list.append(roww)			#2차원 list로 만들어줌
    
youtube_video_list

In [ ]:
import csv
#-*- coding: utf-8 -*-
...

csvfile = open("C:\\Users\\jmlee\\Code\\amatta.txt","w",-1,"utf-8")
csvwriter = csv.writer(csvfile)
for row in youtube_video_list:
    csvwriter.writerow(row)
csvfile.close()

# 후처리

In [5]:
#python3 이용
import os 
import sys
os.environ["SPARK_HOME"]=os.path.join(os.path.expanduser("~"),"Downloads\spark-2.4.2-bin-hadoop2.7")

os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.7-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [6]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [7]:
amattaRdd=spark.sparkContext.textFile(os.path.join("crawling txt","amatta.txt"))

In [8]:
amattaList=amattaRdd.collect()
print(type(amattaList))

<class 'list'>


In [9]:
#/제거
import re
a=list()

for i in amattaList:
    text=re.sub('/',' ',i)
    a.append(text)
    #print(text)
print(a) 

['"바른막국수 무자비하게 매운 맛-육수도 마셔야 제맛! 한우 살치살도 MUKBANG. Makguksu,Grilled beef",amatta아마따,구독자 7.58만명,조회수 10만회,3개월 전,2019 11 21 18:06:31', '', '"마따쥬니어와 떡만둣국,이남장 핵폭탄 오이김치 먹방MUKBANG-Rice cake&dumpling soup",amatta아마따,구독자 7.58만명,조회수 9.5만회,4개월 전,2019 11 21 18:06:31', '', '참치고추장찌개에 흰쌀밥 MUKBANG Korean spicy potato stew,amatta아마따,구독자 7.58만명,조회수 11만회,4개월 전,2019 11 21 18:06:31', '', '매일 먹고 싶은 감자탕❤MUKBANG  Gamjatang(Pork Back-bone Stew),amatta아마따,구독자 7.58만명,조회수 7.7만회,4개월 전,2019 11 21 18:06:31', '', '청양크림우동에 떡국떡 넣어서 먹방MUKBANG Cream sauce udon&rice cake,amatta아마따,구독자 7.58만명,조회수 8.7만회,5개월 전,2019 11 21 18:06:31', '', '당면불고기와 쌈밥 마따베이비와 먹방MUKBANG  Bulgogi,amatta아마따,구독자 7.58만명,조회수 9.2만회,5개월 전,2019 11 21 18:06:31', '', '매운등갈비 핵김치찜에 당면사리 넣어서 먹방MUKBANG Steamed kimchi&back ribs,amatta아마따,구독자 7.58만명,조회수 10만회,5개월 전,2019 11 21 18:06:31', '', '치즈가락떡볶이|순메밀회냉면|무나물꼬막비빔밥-바르다김선생 신메뉴 3종 먹방MUKBANG,amatta아마따,구독자 7.58만명,조회수 6.9만회,5개월 전,2019 11 21 18:06:31', '', '탕화쿵푸 마라탕에 중국당면 먹방MUKBANG Spicy hot pot(Chinese food-MALATANG)麻辣烫,a

In [10]:
#한글만 남기기
b=[]
for j in a:
    text=re.compile('[^ ㄱ-ㅣ가-힣]+') 
    result = text.sub(' ', j)
    b.append(result)
print(b)
print(type(b))
#     text=re.sub('[^\u3131-\u3163\uac00-\ud7ad]+',' ',i)

[' 바른막국수 무자비하게 매운 맛 육수도 마셔야 제맛  한우 살치살도      아마따 구독자  만명 조회수  만회 개월 전       ', '', ' 마따쥬니어와 떡만둣국 이남장 핵폭탄 오이김치 먹방     아마따 구독자  만명 조회수  만회 개월 전       ', '', '참치고추장찌개에 흰쌀밥          아마따 구독자  만명 조회수  만회 개월 전       ', '', '매일 먹고 싶은 감자탕        아마따 구독자  만명 조회수  만회 개월 전       ', '', '청양크림우동에 떡국떡 넣어서 먹방         아마따 구독자  만명 조회수  만회 개월 전       ', '', '당면불고기와 쌈밥 마따베이비와 먹방    아마따 구독자  만명 조회수  만회 개월 전       ', '', '매운등갈비 핵김치찜에 당면사리 넣어서 먹방       아마따 구독자  만명 조회수  만회 개월 전       ', '', '치즈가락떡볶이 순메밀회냉면 무나물꼬막비빔밥 바르다김선생 신메뉴  종 먹방 아마따 구독자  만명 조회수  만회 개월 전       ', '', '탕화쿵푸 마라탕에 중국당면 먹방         아마따 구독자  만명 조회수  만회 개월 전       ', '', '불닭쫄볶이에 핵불닭떡볶이 섞어서 초란후라이랑 먹방             아마따 구독자  만명 조회수  만회 개월 전       ', '', ' 매콤 가리비 키조개 닭찜 먹방 당면 콩나물도 숨어있어요             아마따 구독자  만명 조회수  만회 개월 전       ', '', ' 호미불닭발 무뼈닭발과 오돌뼈세트에 동그랑땡 먹방           아마따 구독자  만명 조회수  만회 개월 전       ', '', '두찜 찜닭 매운맛  단계 맛있당 먹방           아마따 구독자  만명 조회수  만회 개월 전       ', '', '매운 김치볶음밥 날치알 듬뿍 넣고 먹방       아마따 구독자  만명 조회수  만회 개월 전       ', '', ' 치즈핵불닭

In [11]:
amattaRdd_=spark.sparkContext.parallelize(b)  

In [12]:
#불용어제거하고 빈도수 추출
stopwords = [u'전',u'추가',u'제맛',u'넣은',u'천회',u'조회수',u'개월',u'년',u'퍼먹는',u'퍼먹기',u'같이',u'종',u'아마따',u'넣고',u'아마따표',u'마따베이비',u'아마따구독자',u'천회년',u'만회년',u'먹방아마따구독자',u'만회년',u'먹방',u'만명조회수',u'리얼사운드',u'구독자',u'만회일',u'먹고',u'만회개월',u'마따베이비와',u'넣어서',u'만명',u'만회',u'듬뿍',u'많이',u'매운맛',u'특매운맛',u'밥도',u'무섭게',u'한신포차']
amattastop=amattaRdd_.flatMap(lambda x:x.split()).filter(lambda x:x not in stopwords)
print(type(amattastop))

<class 'pyspark.rdd.PipelinedRDD'>


In [13]:
amattastop=amattastop.collect()
print(amattastop)
print(type(amattastop))
#' '제거하는법

['바른막국수', '무자비하게', '매운', '맛', '육수도', '마셔야', '한우', '살치살도', '마따쥬니어와', '떡만둣국', '이남장', '핵폭탄', '오이김치', '참치고추장찌개에', '흰쌀밥', '매일', '싶은', '감자탕', '청양크림우동에', '떡국떡', '당면불고기와', '쌈밥', '매운등갈비', '핵김치찜에', '당면사리', '치즈가락떡볶이', '순메밀회냉면', '무나물꼬막비빔밥', '바르다김선생', '신메뉴', '탕화쿵푸', '마라탕에', '중국당면', '불닭쫄볶이에', '핵불닭떡볶이', '섞어서', '초란후라이랑', '매콤', '가리비', '키조개', '닭찜', '당면', '콩나물도', '숨어있어요', '호미불닭발', '무뼈닭발과', '오돌뼈세트에', '동그랑땡', '두찜', '찜닭', '단계', '맛있당', '매운', '김치볶음밥', '날치알', '치즈핵불닭', '키리모찌', '당면볶이', '명란계란찜', '얼큰칼칼', '부대찌개에', '라면사리', '밥', '뚝딱', '초밥과', '새우튀김', '게살크림고로케', '냉모밀', '콩국수와', '낙지덮밥은', '환상의', '짝꿍', '오레오오즈', '통통한', '나드리쫄면과', '더블치즈돈까스', '크림카레우동과', '핵김치', '반짝출연', '매콤한', '도가니', '볶음에', '당면도', '비벼서', '매운', '해물찜', '먹다가', '결국엔', '소스도', '다', '엽떡', '우동사리', '명랑핫도그', '마카롱', '뚱카롱', '짜장밥과', '이남장설렁탕의', '폭탄김치', '매운', '돼지김치찌개에', '당면사리', '흰쌀밥', '뿌링클순살', '빨강소떡', '치즈볼', '핵불닭', '참치비빔밥', '날치알', '크게', '명란두부국', '핵', '뜨거운', '우족탕과', '핵', '매운', '핵폭탄김치', '캡사이신', '양념게장당면볶이', '굵은당면', '떡당면', '훠궈면', '핵김치', '폭탄김치와', '초간단', '누룽지백숙', '넘', '잘먹는', '사골', '

In [14]:
amattastopRdd=spark.sparkContext.parallelize(amattastop) 

In [17]:
#정규화한다음 빈도수 추출
amatta=amattastopRdd\
    .flatMap(lambda x:x.split(','))\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(25)

In [18]:
print(amatta)

[(16, '매운'), (6, '중국당면'), (6, '청양고추'), (5, '전주진김치'), (4, '매콤한'), (4, '김치'), (4, '금치김치'), (3, '떡만둣국'), (3, '당면사리'), (3, '엽떡'), (3, '떡당면'), (3, '훠궈면'), (3, '파김치'), (3, '얼큰한'), (3, '깍두기'), (3, '중국당면도'), (3, '실비김치'), (3, '금치'), (3, '송주불냉면'), (2, '한우'), (2, '흰쌀밥'), (2, '쌈밥'), (2, '마라탕에'), (2, '날치알'), (2, '치즈핵불닭')]


In [20]:
f=open("C:\\Users\\jmlee\\Code\\Frequency.txt",'a',-1,"utf-8")
A=len(amatta)
print (A)
for i in range (0,A):
    f.writelines(str(amatta[i]))
    f.writelines("\n")
f.close()

25
